# AICore bridge - quick test & guide

The AICore Bridge for Stactic's AICore system is a 'boilerplate' module 
that provides an interface to Wodan processor modules. This allows us to
use the same modules through AICore as in Wodan.

## Feature comparision

Wodan provides a rich environment for processor modules that handles data 
retrieval and preprocessing as well as postprocessing of the processor result. 
Not all of this can be ported to AICore - most notably the data retrieval - and 
some still has

We can split Wodan's pre- and post-processing roughly in four stages;
data retrieval, pre-processing, post-processing and formatting.  

### Data retrieval

Data retrieval is done by Wodan and never by AICore, which recieves sample
data together with the request. Still some Wodan functionality can be ported.

In [ ]:
#| echo: false
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

from IPython.display import display
from IPython.display import Markdown
from tabulate import tabulate

def feature_table(feature_dict):
    display(Markdown(tabulate(
      [[v for v in row.values()] for row in feature_dict],
      headers=[k for k in feature_dict[0].keys()],
        tablefmt='github'
    )))
                  

In [ ]:
#| echo: false
feat_matrix_retrieval = [
    dict(feat='truncate', description='bucketize timeseries data on time',
         Wodan=True, AICore=False, remarks='can be implemented'),
    dict(feat='calibrations', description='calibrates or corrects sample data',
         Wodan=True, AICore=False, 
         remarks='can be implemented'),
    dict(feat='corrections', description='exactly same as calibrations',
         Wodan=True, AICore=False, 
         remarks='can be implemented'),
    dict(feat='time format', description='specifies the format for timestamps',
         Wodan=True, AICore=False, 
         remarks='--'),
    dict(feat='timezone', description='specifies the timezone for processing',
         Wodan=True, AICore=False, 
         remarks='only applies to timeFormat="iso"'),
    dict(feat='last seen', description='retrieves a period before the given date',
         Wodan=True, AICore=False, 
         remarks='never, AICore receives data with request'),
    dict(feat='data caching', description='cache data with similar retrieval parameters',
         Wodan=True, AICore=False, 
         remarks='probably never, pointless'),
]

#pd.DataFrame.from_dict(feat_matrix_retrieval).set_index('feat')
feature_table(feat_matrix_retrieval)

| feat         | description                                  | Wodan   | AICore   | remarks                                  |
|--------------|----------------------------------------------|---------|----------|------------------------------------------|
| truncate     | bucketize timeseries data on time            | True    | False    | can be implemented                       |
| calibrations | calibrates or corrects sample data           | True    | False    | can be implemented                       |
| corrections  | exactly same as calibrations                 | True    | False    | can be implemented                       |
| time format  | specifies the format for timestamps          | True    | False    | --                                       |
| timezone     | specifies the timezone for processing        | True    | False    | only applies to timeFormat="iso"         |
| last seen    | retrieves a period before the given date     | True    | False    | never, AICore receives data with request |
| data caching | cache data with similar retrieval parameters | True    | False    | probably never, pointless                |

### Pre-processing

After retrieval Wodan prepares retrieved data for consumptions by the processor 
module. The feature matrix for pre-processing currently is

In [ ]:
#| echo: false
feat_matrix_pre = [
    dict(feat='metadata', description='combine processor params in one object ',
         Wodan=True, AICore=False, 
         remarks='not possible AICore uses this attribute'),
    dict(feat='historic metadata', description='metadata that changes over time ',
         Wodan=True, AICore=False, 
         remarks='not possible AICore reserves "metadata", use history'),
    dict(feat='history', description='provision for processor parameters that change over time',
         Wodan=True, AICore=False, 
         remarks='can be implemented')
]
feature_table(feat_matrix_pre)

| feat              | description                                              | Wodan   | AICore   | remarks                                              |
|-------------------|----------------------------------------------------------|---------|----------|------------------------------------------------------|
| metadata          | combine processor params in one object                   | True    | False    | not possible AICore uses this attribute              |
| historic metadata | metadata that changes over time                          | True    | False    | not possible AICore reserves "metadata", use history |
| history           | provision for processor parameters that change over time | True    | False    | can be implemented                                   |

### Post-processing

In [ ]:
#| echo: false
feat_matrix_post = [
    dict(feat='samplers', description='applies aggregation to results',
         Wodan=True, AICore=False, 
         remarks='can be implemented'),
    dict(feat='lastSeen', description='returns last record of the result',
         Wodan=True, AICore=True, 
         remarks=''),

]
feature_table(feat_matrix_post)

| feat     | description                       | Wodan   | AICore   | remarks            |
|----------|-----------------------------------|---------|----------|--------------------|
| samplers | applies aggregation to results    | True    | False    | can be implemented |
| lastSeen | returns last record of the result | True    | True     |                    |

### Formatting

Due to AICore's not handling NaN (Not-A-Number) values not all formats are implemented.

In [ ]:
#| echo: false
feat_matrix_format = [
    dict(feat='records', description='list-like; format data as rows of record dictionaries',
         Wodan=True, AICore=True, 
         remarks=''),

    dict(feat='table', description='dict-like; includes schema, compact data',
         Wodan=True, AICore=False, 
         remarks='trouble with NaN'),

    dict(feat='split', description='dict-like, separates index from values',
         Wodan=True, AICore=False, 
         remarks='trouble with NaN'),

    dict(feat='timezone', description='converts result to timezone, only with table format ',
         Wodan=True, AICore=False, 
         remarks=''),

]
feature_table(feat_matrix_format)

| feat     | description                                           | Wodan   | AICore   | remarks          |
|----------|-------------------------------------------------------|---------|----------|------------------|
| records  | list-like; format data as rows of record dictionaries | True    | True     |                  |
| table    | dict-like; includes schema, compact data              | True    | False    | trouble with NaN |
| split    | dict-like, separates index from values                | True    | False    | trouble with NaN |
| timezone | converts result to timezone, only with table format   | True    | False    |                  |

### Processors

The following processors have been ported to AICore. Note that any Wodan processor 
can be ported.

In [ ]:
#| echo: false
aicore_modules = dict(
    read = 'e88bf4da-20bc-436a-a867-09c898fc81fe',
    sapflow='d948aa27-601d-41bc-98fd-5241180544c6',
    watergifte = '28f15e05-7542-43e5-baa3-2072f4c7d951',
    dendrometrics = '469e2b8e-2922-410b-863d-98b58204cdd3',
    meeldauw = '5d96342c-7d28-4fc0-ae8d-6cc0c9fafb09',
    element_availability = 'b956fbc8-2f70-4a7c-bddf-93e397748f92',
    sapflux_prediction = '0a817742-8061-430c-b656-0d603ae222a9'
)

In [ ]:
#| echo: false
feature_table([ 
    dict(processor=k, Wodan=True, AICore=(not not i), moduleId=i)
    for k,i in aicore_modules.items()
])

| processor            | Wodan   | AICore   | moduleId                             |
|----------------------|---------|----------|--------------------------------------|
| read                 | True    | True     | e88bf4da-20bc-436a-a867-09c898fc81fe |
| sapflow              | True    | True     | d948aa27-601d-41bc-98fd-5241180544c6 |
| watergifte           | True    | True     | 28f15e05-7542-43e5-baa3-2072f4c7d951 |
| dendrometrics        | True    | True     | 469e2b8e-2922-410b-863d-98b58204cdd3 |
| meeldauw             | True    | True     | 5d96342c-7d28-4fc0-ae8d-6cc0c9fafb09 |
| element_availability | True    | True     | b956fbc8-2f70-4a7c-bddf-93e397748f92 |
| sapflux_prediction   | True    | True     | 0a817742-8061-430c-b656-0d603ae222a9 |

## Imports and setup

In [ ]:
#| code-fold: true
#| code-summary: "Imports and logging"

# First turn off warnings messing up output
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import importlib
import logging

#from wodan import init_console_logging
from corebridge.core import init_console_logging


Added /home/fenke/repos/wodan to PATH
Loading corebridge.aicorebridge 0.3.4 from /home/fenke/repos/corebridge/corebridge/aicorebridge.py


In [ ]:

syslog = init_console_logging(__name__, logging.DEBUG, timestamp=False)

# logging basic configuration
logging.basicConfig(
    format="%(asctime)s \t%(levelname)s\t%(name)s\t%(message)s",
    datefmt="%Y-%m-%dT%H:%M:%S%z",
    level=logging.DEBUG
)

syslog = logging.getLogger()

In [ ]:
import corebridge
from corebridge.aicorebridge import AICoreModule


### Request parameters


In [ ]:
#| code-fold: true
#| code-summary: "Setup environment"

# setup a getenv wrapper that has some neccesary vars
import localenv
from localenv import localtest as getenv
from wodan.whysor import PGSQLDataSourceWhysor

whysorparams = dict(
    base_url=str(getenv("API_URL")).rstrip("/"),
    headers=dict(authorization=getenv("API_TOKEN"))
)

wodanparams = dict(
    base_url=getenv('WODAN_API'),
    headers=dict(
        authorization=getenv("WODAN_TOKEN"),
        **{'Content-Type': "application/json"}
    )
)
ds = PGSQLDataSourceWhysor(os.getenv('PGSQL_URL'))

aicoreparams = dict(
    base_url=getenv('AICORE_API'),
    headers=dict(
        authorization=f"Bearer {getenv('AICORE_TOKEN')}",
        **{'Content-Type': "application/json"}
    )
)

DEBUG	16444	wodan.pgsqldata	pgsqldata.py	9	Loading Module wodan.pgsqldata
DEBUG	16444	wodan.whysor	whysor.py	29	Loading Module wodan.whysor


### Utility functions

In [ ]:
#| code-fold: true
#| code-summary: "Utility functions"

from functools import reduce

def merge_dicts(x, y):
    i = y.pop('datetimeMeasure')
    s = x.get(i, {})
    s.update(y)
    x[i] = s
    return x

async def get_data_from_wodan(since, until, realm, sensorId, truncate='second', format='json/records', timeFormat='iso'):
    async with aiohttp.ClientSession(**wodanparams) as session:
        req = await session.get(
            "/whysor/read", 
            params=dict(
                since=since,
                until=until,
                realm=realm,
                sensorId=sensorId if isinstance(sensorId, str) else json.dumps(sensorId),
                format=format,
                timeFormat=timeFormat,
                truncate=truncate
            ))

        return await req.json()

async def get_data_from_whysor(sensorId, since, until, realm='whysor', truncate='second'):
    async with aiohttp.ClientSession(**whysorparams) as session:
        since = make_naive_utc(make_datetime(since))
        until = min(make_naive_utc(make_datetime(until)), make_datetime(datetime.datetime.now(pytz.utc)))

        filter = {
            "order": 'datetimeMeasure ASC',
            "limit": 5000,
            "where": {"and":[
                {"datetimeMeasure": {"gte": since.isoformat()} },
                {"datetimeMeasure": {"lt": until.isoformat()} }
            ]}
        }

        if isinstance(sensorId, str):
            async with session.get(
                f"/sensors/{sensorId}/read", 
                params={"filter":json.dumps(filter)}) as request:

                assert request.status == 200, f"Error for sensor: {sensorId}, from: {since.isoformat()} until {until.isoformat()}\n{request.url()}"
                return [dict(datetimeMeasure=S["datetimeMeasure"], value=S["value"]) for S in await request.json()]

        elif isinstance(sensorId, dict):
            combined = {}
            for label, sensor in sensorId.items():

                async with session.get(
                    f"/sensors/{sensor}/read", 
                    params={"filter":json.dumps(filter)}) as request:

                    assert request.status == 200, f"Error {request.status} for sensor: {sensor}, from: {since.isoformat()} until {until.isoformat()}\n{request.url}"
                    reduce(
                        merge_dicts, 
                        [{'datetimeMeasure':S["datetimeMeasure"], label:S["value"]} for S in await request.json()], 
                        combined)

            return [{'datetimeMeasure':T, **S} for T, S in combined.items()]

def write_data(data:pd.DataFrame, name:str):
    data.to_json(os.path.join(os.getcwd(), 'data', name+'.json'), orient='table', indent=3)

## AICore Bridge

### Class definition

In [ ]:
#| code-fold: true
#| code-summary: "AICore Core Module"

from corebridge.aicorebridge import AICoreModule

def test_function(data:pd.DataFrame, anumber:float=0):
    v = 2*anumber
    return data

class TestAICoreModule(AICoreModule):
    def __init__(self, save_dir, *args, **kwargs):
        super().__init__(test_function, save_dir, None, *args, **kwargs)


### Test initialization

In [ ]:
#| code-fold: true
#| code-summary: "Test initialization"

test_module = TestAICoreModule(os.path.join(os.getcwd(), 'cache'), 1, 2, num_1=3, num_2=4)

assert test_module.init_args == (1, 2)
assert test_module.init_kwargs['num_1'] == 3
assert test_module.init_kwargs['num_2'] == 4

### Test inference

In [ ]:
#| code-fold: true
#| code-summary: "Test inference"

test_data = [
    dict(datetimeMeasure='2020-04-01T00:01:11.123Z', value=1.1),
    dict(datetimeMeasure='2020-04-02T00:20:00Z', value=2.3),
]
result = test_module.infer(test_data, timezone='Europe/Amsterdam', anumber=None)
print("Test Data\n", json.dumps(test_data, indent=2))
print("Result Message\n", json.dumps(result['msg'], indent=2))
print("Result Data\n", json.dumps(result['data'], indent=2))

Test Data
 [
  {
    "datetimeMeasure": "2020-04-01T00:01:11.123Z",
    "value": 1.1
  },
  {
    "datetimeMeasure": "2020-04-02T00:20:00Z",
    "value": 2.3
  }
]
Result Message
 [
  "Startup time: 2025-01-20T10:26:41.199526+00:00",
  "Corebridge version: 0.3.4",
  "test_function((data: pandas.core.frame.DataFrame, anumber: float = 0))",
  "init_args: (1, 2), init_kwargs: {'num_1': 3, 'num_2': 4, 'assets_dir': None, 'save_dir': '/home/fenke/repos/wodan/blog/posts/aicore-bridge/cache'}",
  "lastSeen: False, recordformat: records, timezone: Europe/Amsterdam",
  "calldata shape: (2, 1)",
  "anumber: 0.0",
  "result shape: (2, 1)",
  "return-data shape: (2, 1)"
]
Result Data
 [
  {
    "time": "2020-04-01T02:01:11.123000+02:00",
    "value": 1.1
  },
  {
    "time": "2020-04-02T02:20:00+02:00",
    "value": 2.3
  }
]


## Processors

### Read (echo)

#### Wodan read

In [ ]:
#| code-fold: true
#| code-summary: "Getting the moisture data for show"

data_params = dict(
    sensorId='d7a965e8-60c0-4688-a18a-efae93b4a03f',
    since=make_datetime("2023-05-04T10:00:00").isoformat(),
    until=make_datetime("2023-05-04T11:00:00").isoformat(),
)

data_params.update({
    "realm": "whysor", 
    #"truncate":"second"
})

format_params=dict(
    format="json/table",
    timeFormat='iso'
)

call_params=dict(
)

t0=0
async with aiohttp.ClientSession(**wodanparams) as session:
    t0 = time.perf_counter_ns()
    req = await session.get(
        "/whysor/read", 
        params={
            **data_params,
            **format_params
        })
    t1 = time.perf_counter_ns()

    moisturedata = await req.read()
    df_moisture = pd.read_json(
        moisturedata.decode("utf-8"), 
        orient='table')
    jsondata = await req.json()
    print(f"Query took {round((t1-t0) / 1e6,2)} ms for {len(df_moisture)} records")
    print(df_moisture)
    #df_moisture.plot()
    write_data(df_moisture, "read_simple_moisture_data")

Query took 14.47 ms for 3 records
                           value
time                            
2023-05-04 10:04:49+00:00  16.72
2023-05-04 10:24:51+00:00  16.65
2023-05-04 10:44:53+00:00  16.55


#### AICore - direct call

We call the AICore module directly to demonstrate the inner workings

In [ ]:
from wodan.processors.read import Module as Echo

echo_module = Echo(os.path.join(os.getcwd(), 'cache'), None)

AICore call parameters consist of the data and the call parameters in the form a a kwargs property. SensorId can be a dictionary to provide named columns for the analytics functions (it's up to the function to specify what it needs).

In [ ]:
call_params

{}

In [ ]:
#| code-fold: true
#| code-summary: "Watergifte Processor in AICore"
data_params['sensorId'] = dict(soilMoisture=data_params.get('sensorId'))

aicore_call_params = dict(
    data=await get_data_from_whysor(**data_params),
    #data=watergifte_module.rewrite_data(df_moisture.copy(), 
    kwargs=call_params,
)

AssertionError: Error 401 for sensor: d7a965e8-60c0-4688-a18a-efae93b4a03f, from: 2023-05-04T10:00:00+00:00 until 2023-05-04T11:00:00+00:00
https://api.whysor.com/sensors/d7a965e8-60c0-4688-a18a-efae93b4a03f/read?filter=%7B%22order%22:+%22datetimeMeasure+ASC%22,+%22limit%22:+5000,+%22where%22:+%7B%22and%22:+%5B%7B%22datetimeMeasure%22:+%7B%22gte%22:+%222023-05-04T10:00:00%2B00:00%22%7D%7D,+%7B%22datetimeMeasure%22:+%7B%22lt%22:+%222023-05-04T11:00:00%2B00:00%22%7D%7D%5D%7D%7D

The call is made to the *infer* method of the AICore module object.

In [ ]:
t0 = time.perf_counter_ns()
result = echo_module.infer(aicore_call_params['data'], **aicore_call_params['kwargs'])
t1 = time.perf_counter_ns()
print(f"Execution took {round((t1-t0) / 1e6,2)} ms.")


From the AICore module we get a dictionary with the fields

In [ ]:
print([K for K in result.keys()])

In [ ]:
print(json.dumps(result.get('msg'), indent=3))

Where:

msg
: contains the messages from AICore Module

In [ ]:
print("\n".join(result['msg']))

data
: contains the result data from AICore Module processing

In [ ]:
print(json.dumps(result['data'], indent=2))

#### AICore - remote call

For the call to the actual AICore module we add a format parameter to define how dataframe is formatted

In [ ]:
#| code-fold: true
#| code-summary: "Watergifte Processor in AICore"


format_params=dict(
    format="records",
)
print(json.dumps(format_params, indent=2))

We make the call, read the response into a dataframe and convert it to a numpy array for the plot. The 
online call is made to the *post* method of the AICore, specifying the company and project in the URL and
the call parameters in the body. The body is a list of call parameters where each object is a dictionary specifying:


In [ ]:
t00 = time.perf_counter_ns()

aicore_call_params = dict(
        module_id=aicore_modules['read'],
        kwargs=dict(**call_params, **format_params),
        data=await get_data_from_whysor(**data_params),
    )

t01 = time.perf_counter_ns()

The *aicore_call_params* looks basicaly like this:

In [ ]:
print(json.dumps({k:v[0:2] if isinstance(v, list) else v for k,v in aicore_call_params.items()}, indent=2))

module_id
: the id of the module to call

kwargs
: the call parameters for the module

data
: the data to pass to the module - it's expected to be formatted according to the format parameter

We now could make the REST call to the AICore module

In [ ]:
async with aiohttp.ClientSession(**aicoreparams) as session:
    t10 = time.perf_counter_ns()

    req = await session.post(
        "/api/infer/", 
        #params=dict(company='whysor', project='aicorebridge'),
        data = json.dumps(aicore_call_params, indent=2))
    t11 = time.perf_counter_ns()

    readdata = await req.read()
    raw_read = json.loads(readdata.decode("utf-8"))

    print(f"Query took {round((t11-t10 + (t01-t00)) / 1e6,2)} ms (whysor time: {round((t01-t00) / 1e6,2)} + aicore time: {round((t10-t11) / 1e6,2)}) ")

### Watergifte

#### Moisture data

In [ ]:
#| code-fold: true
#| code-summary: "Getting the moisture data for show"

data_params = dict(
    sensorId='d7a965e8-60c0-4688-a18a-efae93b4a03f',
    since=make_datetime("2023-05-04").isoformat(),
    until=make_datetime("2023-08-08").isoformat(),
)

data_params.update({
    "realm": "whysor", 
    "truncate":"second"
})

format_params=dict(
    format="json/split-index",
    timeFormat='iso'
)

call_params=dict(
    sensitivity=4.0
)

t0=0
async with aiohttp.ClientSession(**wodanparams) as session:
    t0 = time.perf_counter_ns()
    req = await session.get(
        "/whysor/read", 
        params={
            **data_params,
            **format_params
        })
    t1 = time.perf_counter_ns()

    moisturedata = await req.read()
    df_moisture = pd.read_json(
        moisturedata.decode("utf-8"), 
        orient='split')
    jsondata = await req.json()
    print(f"Query took {round((t1-t0) / 1e6,2)} ms for {len(df_moisture)} records")

    df_moisture.plot()
    write_data(df_moisture, "basic_moisture_data")

#### Wodan

Base parameters for the query determine the formatting of the result data

In [ ]:
format_params=dict(
    format="json/split-index",
    timeFormat='iso'
)

De call parameters specific to the Watergifte module, there are more options but leave it at *sensitivity* and we use the defaults for the others.

In [ ]:
call_params=dict(
    sensitivity=5.0
)

For the REST call we combine the parameters into the url parameters

In [ ]:
url_params=dict(
    **data_params,
    **call_params,
    **format_params
)

We make the call, read the response into a dataframe and convert it to a numpy array for the plot

In [ ]:
t0=0
async with aiohttp.ClientSession(**wodanparams) as session:
    t0 = time.perf_counter_ns()
    req = await session.get(
        "/whysor/watergifte", 
        params=url_params)
    t1 = time.perf_counter_ns()

    wateringdata = await req.read()
    df_watering_wodan = pd.read_json(
        wateringdata.decode("utf-8"), 
        orient='split')
    jsondata = await req.json()
    print(f"Query took {round((t1-t0) / 1e6,2)} ms for {len(df_watering_wodan)} records")
    write_data(df_watering_wodan, "basic_moisture_moments")


    vlines=df_watering_wodan.index.to_numpy()
    df_moisture.plot().vlines(vlines, ymin=df_moisture.min(), ymax=df_moisture.max(), linestyles='dotted')

#### AICore - direct call

We call the AICore module directly to demonstrate the inner workings

In [ ]:
from wodan.processors.watergifte import Module as WatergifteModule

watergifte_module = WatergifteModule(os.path.join(os.getcwd(), 'cache'), None)

AICore call parameters consist of the data and the call parameters in the form a a kwargs property. SensorId can be a dictionary to provide named columns for the analytics functions (it's up to the function to specify what it needs).

In [ ]:
#| code-fold: true
#| code-summary: "Watergifte Processor in AICore"
data_params['sensorId'] = dict(soilMoisture=data_params.get('sensorId'))

aicore_call_params = dict(
    data=await get_data_from_whysor(**data_params),
    #data=watergifte_module.rewrite_data(df_moisture.copy(), 
    kwargs=call_params,
)

The call is made to the *infer* method of the AICore module object.

In [ ]:
t0 = time.perf_counter_ns()
result = watergifte_module.infer(aicore_call_params['data'], **aicore_call_params['kwargs'])
t1 = time.perf_counter_ns()
print(f"Execution took {round((t1-t0) / 1e6,2)} ms.")


From the AICore module we get a dictionary with the fields

In [ ]:
print([K for K in result.keys()])

Where:

msg
: contains the messages from AICore Module

In [ ]:
print("\n".join(result['msg']))

data
: contains the result data from AICore Module processing

In [ ]:
print(json.dumps(result['data'], indent=2))

#### AICore - remote call

For the call to the actual AICore module we add a format parameter to define how dataframe is formatted

In [ ]:
#| code-fold: true
#| code-summary: "Watergifte Processor in AICore"


format_params=dict(
    format="records",
)
print(json.dumps(format_params, indent=2))

We make the call, read the response into a dataframe and convert it to a numpy array for the plot. The 
online call is made to the *post* method of the AICore, specifying the company and project in the URL and
the call parameters in the body. The body is a list of call parameters where each object is a dictionary specifying:


In [ ]:
t00 = time.perf_counter_ns()
aicore_call_params = dict(
        module_id = aicore_modules['watergifte'],
        kwargs=dict(**call_params, **format_params),
        data=await get_data_from_whysor(**data_params),
    )

t01 = time.perf_counter_ns()

The *aicore_call_params* looks basicaly like this:

In [ ]:
print(json.dumps({k:v[0:5] if isinstance(v, list) else v for k,v in aicore_call_params.items()}, indent=2))


kwargs
: the call parameters for the module

data
: the data to pass to the module - it's expected to be formatted according to the format parameter

We now make the REST call to the AICore module and read the response

In [ ]:
async with aiohttp.ClientSession(**aicoreparams) as session:
    t10 = time.perf_counter_ns()

    req = await session.post(
        "/api/infer/", 
        data = json.dumps(aicore_call_params))
    t11 = time.perf_counter_ns()

    wateringdata = await req.read()
    raw_watering = json.loads(wateringdata.decode("utf-8"))

    print(f"Query took {round((t11-t10 + (t01-t00)) / 1e6,2)} ms (whysor time: {round((t01-t00) / 1e6,2)} + aicore time: {round((t10-t11) / 1e6,2)}) ")

The cloud call returns a multi-level dictionary with a 'modules' key where it's value is a dictionary with module id's as keys, and each associated value is a dict with keys msg and data that were returned by the infer method of the CustomModule. We get the first module-id in the list and like before we get a dictionary with keys:

In [ ]:
raw_watering

In [ ]:
list(raw_watering.keys())

msg
: messages from AICore Module

In [ ]:
print("\n".join(raw_watering['msg']))

data
: data from AICore Module

In [ ]:
print(json.dumps(raw_watering['data'], indent=2))

For pandas data frame we can use

In [ ]:
#| code-fold: true
df_watering = pd.DataFrame.from_dict(
    raw_watering['data'])
df_watering.set_index('time', inplace=True)
df_watering.index = pd.to_datetime(df_watering.index)

print(df_watering)

### Sapflow {#sec-function-sapflow}

#### Wodan

In [ ]:
#| code-fold: true
#| code-summary: "Sapflow Processor"

sensorId = dict(
    uncorrectedOuter='291fffc3-790a-40b4-91e3-bc720d254ede',
    uncorrectedInner='1d28e24c-f6d8-4ffc-b23d-7c08c6f803d2',
)

data_params = dict(
    since=make_datetime("2023-09-01").isoformat(),
    until=make_datetime("2023-09-08").isoformat(),
    realm="whysor", 
    truncate="second"
)

format_params=dict(
    format="json/table",
    timeFormat='iso'
)

# For in-depth testing we write the basic data 

async with aiohttp.ClientSession(**wodanparams) as session:
    t0 = time.perf_counter_ns()
    req = await session.get(
        "/whysor/read", 
        params=dict(
            sensorId=json.dumps(sensorId),
            **data_params,
            **format_params
        ))
    t1 = time.perf_counter_ns()

    flowdataraw = await req.read()
    df_flowdata = pd.read_json(
        flowdataraw.decode("utf-8"), 
        orient='table')
    print(f"Query took {round((t1-t0) / 1e6,2)} ms for {len(df_flowdata)} records")
    print(df_flowdata)
    #df_moisture.plot()
    write_data(df_flowdata, "basic_sapflow_data")

In [ ]:
call_params=dict(
    timezone='Europe/London',
)

#| code-fold: true
payload=[
    dict(
        **data_params,
        sensorId = sensorId,
        metadata=dict(
            outlierSensitivity = 4,
            flowFocus='Outer and inner (mean)',
            barkDepth=0.5,
            circumference=19.8
        )
    ),
    
]
t0=0
async with aiohttp.ClientSession(**wodanparams) as session:
    t0 = time.perf_counter_ns()
    req = await session.post(
        "/whysor/sapflow", 
        params={
            **format_params,
            **call_params
        },
        data=json.dumps(payload)
    )
    t1 = time.perf_counter_ns()
    if req.status == 200:
        rdata = await req.json()
        if rdata:
            print('resolved request:', json.dumps(rdata[0]['request'],indent=3))
            df_sapflow = pd.read_json(
                json.dumps(rdata[0]['data']), 
                orient='table')
            print(df_sapflow.info())
            
            print(df_sapflow.head(5))
            df_sapflow[[c for c in df_sapflow.columns if 'Volume' in c]].plot()
        
            write_data(df_sapflow, "basic_sapflow_result_data")

    print(f"Query took {round((t1-t0) / 1e6,2)} ms")

In [ ]:
df_sapflow.isna().any(axis=None)

#### AICore - direct call

In [ ]:
#| code-fold: true
import wodan.processors.sapflow
importlib.reload(wodan.processors.sapflow)

from wodan.processors.sapflow import Module as SapflowModule
sapflow_module = SapflowModule(None, None)

In [ ]:
data_params, format_params

In [ ]:
#| code-fold: true
#| code-summary: "Sapflow Processor in AICore"
metadata=dict(
    outlierSensitivity = 4,
    flowFocus='Outer and inner (mean)',
    barkDepth=0.5,
    circumference=19.8
)

aicore_call_params = dict(
    kwargs=dict(
        **call_params, 
        format="records",
        **metadata),

    data=await get_data_from_wodan(**data_params, sensorId=sensorId, format="json/records")
)

t0 = time.perf_counter_ns()
result = sapflow_module.infer(aicore_call_params['data'], **aicore_call_params['kwargs'])
t1 = time.perf_counter_ns()
print(f"Execution took {round((t1-t0) / 1e6,2)} ms.")

#print(json.dumps(result, indent=2))

From the AICore module we get a dictionary with the fields

In [ ]:
print([K for K in result.keys()])

msg
: messages from AICore Module

In [ ]:
print("\n".join(result['msg']))

data
: data from AICore Module

In [ ]:
print(json.dumps(result['data'][0:6], indent=2))

#### AICore - remote call

In [ ]:
#| code-fold: true
#| code-summary: "Watergifte Processor in AICore"


format_params=dict(
    format="records",
)


t0=0
async with aiohttp.ClientSession(**aicoreparams) as session:
    t0 = time.perf_counter_ns()
    
    # simulate remote call including fetching data
    aicore_call_params = dict(
            module_id=aicore_modules['sapflow'],
            kwargs=dict(**call_params, **format_params, **metadata),
            data=await get_data_from_wodan(**data_params, sensorId=sensorId)
        )
    
    t2 = time.perf_counter_ns()

    req = await session.post(
        "/api/infer/", 
        data = json.dumps(aicore_call_params))
    t1 = time.perf_counter_ns()

    sapflowdata = await req.read()
    try:
        raw_sapflow = json.loads(sapflowdata.decode("utf-8"))
    except:
        print(sapflowdata.decode("utf-8"))
        raw_sapflow = None
        pass
    print(f"Query took {round((t1-t0) / 1e6,2)} ms (whysor time: {round((t2-t0) / 1e6,2)} + aicore time: {round((t1-t2) / 1e6,2)}) for {0} records")

The *aicore_call_params* that we send to the AICore module looks basicaly like this:

In [ ]:
print(json.dumps({k:v[0:5] if isinstance(v, list) else v for k,v in aicore_call_params.items()}, indent=2))

The result is a dictionary containing:

msg
: messages from AICore Module

In [ ]:
if raw_sapflow:
    print("\n".join(raw_sapflow['msg']))

data
: data from AICore Module

In [ ]:
if raw_sapflow:
    print(json.dumps(raw_sapflow['data'][0:3], indent=2))

For pandas data frame we can use

In [ ]:
#| code-fold: true
if raw_sapflow:

    df_sapflow = pd.DataFrame.from_dict(
        raw_sapflow['data'])
    df_sapflow.set_index('time', inplace=True)
    df_sapflow.index = pd.to_datetime(df_sapflow.index, format='ISO8601')

    print(df_sapflow)

### Dendometrics {#sec-function-dendrometrics}

#### Wodan


In [ ]:
#| code-fold: true
#| code-summary: "Sapflow Processor"
sensorId = 'f096c2fa-43fe-402d-a982-7868f6840277'

data_params = dict(
    since=make_datetime("2023-09-01").isoformat(),
    until=make_datetime("2023-09-08").isoformat(),
    realm="whysor", 
    truncate="second"
)

format_params=dict(
    format="json/table",
    timeFormat='iso'
)

# For in-depth testing we write the basic data 

async with aiohttp.ClientSession(**wodanparams) as session:
    t0 = time.perf_counter_ns()
    req = await session.get(
        "/whysor/read", 
        params=dict(
            sensorId=sensorId,
            **data_params,
            **format_params
        ))
    t1 = time.perf_counter_ns()

    dendrodataraw = await req.read()
    df_dendrodata = pd.read_json(
        dendrodataraw.decode("utf-8"), 
        orient='table')
    print(f"Query took {round((t1-t0) / 1e6,2)} ms for {len(df_dendrodata)} records")
    print(df_dendrodata)
    #df_moisture.plot()
    write_data(df_dendrodata, "basic_dendro_data")

In [ ]:
call_params=dict(
    timezone='Europe/London',
)

#| code-fold: true
payload=[
    dict(
        **data_params,
        sensorId = sensorId,
    ),
    
]
t0=0
async with aiohttp.ClientSession(**wodanparams) as session:
    t0 = time.perf_counter_ns()
    req = await session.post(
        "/whysor/dendrometrics", 
        params={
            **format_params,
            **data_params
        },
        data=json.dumps(payload)
    )
    t1 = time.perf_counter_ns()

    if req.status == 200:
        rdata = await req.json()
        if rdata:
            print('resolved request:', json.dumps(rdata[0]['request'],indent=3))
            df_dendro = pd.read_json(
                json.dumps(rdata[0]['data']), 
                orient='table')
            print(df_dendro.info())
            
            print(df_dendro.tail(5))
            df_dendro[[c for c in df_dendro.columns if 'clean' not in c]].plot()
        
            write_data(df_dendro, "basic_sapflow_result_data")

    print(f"Query took {round((t1-t0) / 1e6,2)} ms")

#### AICore - remote call


In [ ]:
#| code-fold: true
#| code-summary: "Watergifte Processor in AICore"


format_params=dict(
    format="records",
)


t0=0
async with aiohttp.ClientSession(**aicoreparams) as session:
    t0 = time.perf_counter_ns()
    
    # simulate remote call including fetching data
    aicore_call_params = dict(
            module_id=aicore_modules['dendrometrics'],
            kwargs=dict(**format_params),
            data=await get_data_from_whysor(**data_params, sensorId=sensorId)
        )
    
    t2 = time.perf_counter_ns()

    req = await session.post(
        "/api/infer/", 
        data = json.dumps(aicore_call_params))
    t1 = time.perf_counter_ns()

    dendrodata = await req.read()
    try:
        raw_dendro = json.loads(dendrodata.decode("utf-8"))
    except:
        print(dendrodata.decode("utf-8"))
        raw_dendro = None
        pass
    print(f"Query took {round((t1-t0) / 1e6,2)} ms (whysor time: {round((t2-t0) / 1e6,2)} + aicore time: {round((t1-t2) / 1e6,2)})")

The call request then looks like:

In [ ]:
print(json.dumps({k:v[0:5] if isinstance(v, list) else v for k,v in aicore_call_params.items()}, indent=2))

The resulting data

In [ ]:
print(json.dumps({k:v[0:5] if isinstance(v, list) else v for k,v in raw_dendro.items()}, indent=2))

Interpreted as dataframe the result

In [ ]:
#| code-fold: true
if raw_dendro:

    df_dendro = pd.DataFrame.from_dict(
        raw_dendro['data'])
    df_dendro.set_index('time', inplace=True)
    df_dendro.index = pd.to_datetime(df_dendro.index, format='ISO8601')

    print(df_dendro.tail(5))
    df_dendro[[c for c in df_dendro.columns if 'clean' not in c]].plot()


### Element availability

In [ ]:
soillife_sensors = dict(
    redoxPotential = 'cc0787a2-c7a4-4b33-ba9f-1974cc2483b5',
    acidity = '6f178072-020a-4b45-b4de-410665fda7db'
)
element_sensors = dict(
    Cu='af118fab-2e59-450d-98bd-805a45c627b7',
    Mn='b02c223f-0b2a-46a3-8771-5286721e4ccc'
)

In [ ]:
async with aiohttp.ClientSession(**whysorparams) as session:
    for I in set(soillife_sensors.values()) | set(element_sensors.values()):
        await ingest_whysordata(ds, session, **dict(
            sensorId=I,
            realm='whysor',
            since='2022-01-01',
            until=datetime.datetime.utcnow().isoformat()
        ))


#### Input data

In [ ]:
data_params = dict(
    since='2024-01-01',
    until='2024-02-01',
)

#### Wodan

In [ ]:
format_params=dict(
    format="json/table",
    timeFormat='iso'
)

# For in-depth testing we write the basic data 

async with aiohttp.ClientSession(**wodanparams) as session:
    t0 = time.perf_counter_ns()
    req = await session.get(
        "/whysor/element_availability", 
        params=dict(
            sensorId=json.dumps(soillife_sensors),
            **data_params,
            **format_params
        ))
    t1 = time.perf_counter_ns()

    elementdataraw = await req.read()
    df_elementdata = pd.read_json(
        elementdataraw.decode("utf-8"), 
        orient='table')
    print(f"Query took {round((t1-t0) / 1e6,2)} ms for {len(df_elementdata)} records")
    print(df_elementdata)

#### AICore

In [ ]:
#| code-fold: true
#| code-summary: "Watergifte Processor in AICore"


format_params=dict(
    format="records",
)


t0=0
async with aiohttp.ClientSession(**aicoreparams) as session:
    t0 = time.perf_counter_ns()
    
    # simulate remote call including fetching data
    aicore_call_params = dict(
            module_id=aicore_modules['element_availability'],
            kwargs=dict(**format_params),
            data=await get_data_from_whysor(**data_params, sensorId=soillife_sensors.copy())
        )
    
    t2 = time.perf_counter_ns()

In [ ]:
print(json.dumps({k:v[0:5] if isinstance(v, list) else v for k,v in aicore_call_params.items()}, indent=2))

In [ ]:
t01 = time.perf_counter_ns()
async with aiohttp.ClientSession(**aicoreparams) as session:

    req = await session.post(
        "/api/infer/", 
        data = json.dumps(aicore_call_params))
    t1 = time.perf_counter_ns()

    elementdata = await req.read()
    try:
        raw_elementdata = json.loads(elementdata.decode("utf-8"))
    except:
        print(elementdata.decode("utf-8"))
        raw_elementdata = None
        pass
    print(f"Query took {round((t1-t01+t2-t0) / 1e6,2)} ms (whysor time: {round((t2-t0) / 1e6,2)} + aicore time: {round((t1-t01) / 1e6,2)}) for {0} records")

In [ ]:
print('\n'.join(raw_elementdata['msg']))

In [ ]:
if raw_elementdata:
    print(json.dumps(raw_elementdata['data'][0:2], indent=2))

In [ ]:
#| code-fold: true
if raw_elementdata:

    df_elementdata = pd.DataFrame.from_dict(
        raw_elementdata['data'])
    df_elementdata.set_index('time', inplace=True)
    df_elementdata.index = pd.to_datetime(df_elementdata.index, format='ISO8601')

    print(df_elementdata)

### Mildew



In [ ]:
import pandas as pd

In [ ]:
pd.read_json('data/testdata_meeldauw.json', orient='records').head().to_dict(orient='records')

In [ ]:
#| code-fold: true
#| code-summary: "Mildew Processor in AICore"


format_params=dict(
    format="records",
)
call_params = dict()
metadata = dict()

t0=0
async with aiohttp.ClientSession(**aicoreparams) as session:
    t0 = time.perf_counter_ns()
    
    # simulate remote call including fetching data
    aicore_call_params = dict(
            module_id=aicore_modules['meeldauw'],
            kwargs=dict(**call_params, **format_params, **metadata),
            data=[
                {k:v for k,v in R.items() if v is not None and not pd.isna(v)}
                for R in pd.read_json('data/testdata_meeldauw.json', orient='records').to_dict(orient='records')
            ]
        )
    
    t2 = time.perf_counter_ns()

    req = await session.post(
        "/api/infer/", 
        data = json.dumps(aicore_call_params))
    t1 = time.perf_counter_ns()

    mildewdata = await req.read()
    try:
        raw_mildewdata = json.loads(mildewdata.decode("utf-8"))
    except:
        print(mildewdata.decode("utf-8"))
        raw_mildewdata = None
        pass
    print(f"Query took {round((t1-t0) / 1e6,2)} ms (whysor time: {round((t2-t0) / 1e6,2)} + aicore time: {round((t1-t2) / 1e6,2)}) for {0} records")

The payload looks like:

In [ ]:
print(json.dumps({k:v[0:5] if isinstance(v, list) else v for k,v in aicore_call_params.items()}, indent=2))

The returned messages can be found und the 'msg' key

In [ ]:
#| echo: true
print("\n".join(raw_mildewdata['msg']))

In [ ]:
pd.DataFrame(raw_mildewdata['data']).set_index('time').head(9)

#### Product gebruik

With the parameter `productgebruik` set to `True` de module will calculate the cumlative use of each product from the
input data and return a table with used and remaining uses for each product. 

In [ ]:
#| code-fold: true
#| code-summary: "Mildew Processor in AICore"


call_params=dict(
    productgebruik=True,
)

The module only needs the rows with `product` use

In [ ]:
pd.read_json('data/testdata_meeldauw.json', orient='records')[['time', 'product']].dropna()

In [ ]:
t0=0
async with aiohttp.ClientSession(**aicoreparams) as session:
    t0 = time.perf_counter_ns()
    
    # simulate remote call including fetching data
    aicore_call_params = dict(
            module_id=aicore_modules['meeldauw'],
            kwargs=dict(**call_params, **format_params, **metadata),
            data=[
                {k:v for k,v in R.items() if v is not None and not pd.isna(v)}
                for R in pd.read_json('data/testdata_meeldauw.json', orient='records')[['time', 'product']].dropna().to_dict(orient='records')
            ]
        )
    
    t2 = time.perf_counter_ns()

    req = await session.post(
        "/api/infer/", 
        data = json.dumps(aicore_call_params))
    t1 = time.perf_counter_ns()

    mildewdata = await req.read()
    try:
        raw_mildewdata = json.loads(mildewdata.decode("utf-8"))
    except:
        print(mildewdata.decode("utf-8"))
        raw_mildewdata = None
        pass
    print(f"Query took {round((t1-t0) / 1e6,2)} ms (whysor time: {round((t2-t0) / 1e6,2)} + aicore time: {round((t1-t2) / 1e6,2)}) for {0} records")

The payload passed to the module looks like

In [ ]:
#|echo: false
print(json.dumps(
    {
        k:v[0:5] if isinstance(v, list) else v 
        for k,v in aicore_call_params.items()}, 
    indent=2
))

dan wordt een tabel geretourneerd met reeds gebruikte en de nog resterende gebruiksmgelijkheden per product.

In [ ]:
#|echo: false
print(json.dumps(raw_mildewdata['data'][0:3], indent=2 ))

In [ ]:
pd.DataFrame(raw_mildewdata['data'])

## RScript bridge

### Remote call

In [ ]:
#| code-fold: true
#| code-summary: "Sapflow Prediction RScript Processor in AICore"


t0=0
async with aiohttp.ClientSession(**aicoreparams) as session:
    t0 = time.perf_counter_ns()
    
    # simulate remote call including fetching data
    aicore_call_params = dict(
            module_id=aicore_modules['sapflux_prediction'],
            kwargs=dict(
                format='records',
                readTag='Predicted_sapflux',
                camelCase=True
            ),
            data=[]
        )
    
    t2 = time.perf_counter_ns()

    req = await session.post(
        "/api/infer/", 
        data = json.dumps(aicore_call_params))
    t1 = time.perf_counter_ns()

    sapflowdata = await req.read()
    try:
        raw_sapflow = json.loads(sapflowdata.decode("utf-8"))
    except:
        print(sapflowdata.decode("utf-8"))
        raw_sapflow = None
        pass
    print(f"Query took {round((t1-t0) / 1e6,2)} ms (whysor time: {round((t2-t0) / 1e6,2)} + aicore time: {round((t1-t2) / 1e6,2)}) for {0} records")

In [ ]:
print(json.dumps({k:v[0:5] if isinstance(v, list) else v for k,v in aicore_call_params.items()}, indent=2))

In [ ]:
if raw_sapflow:
    print("\n".join(raw_sapflow['msg']))

In [ ]:
if raw_sapflow:
    print(json.dumps(raw_sapflow['data'][0:3], indent=2))

In [ ]:
#| code-fold: true
if raw_sapflow:

    df_sapflow = pd.DataFrame.from_dict(
        raw_sapflow['data'])
    df_sapflow.set_index('time', inplace=True)
    df_sapflow.index = pd.to_datetime(df_sapflow.index, format='ISO8601')

    print(df_sapflow)